# 1. Import Libraries:

In [27]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 2. Define Image Size and Batch Size:

In [28]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 64  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [29]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

# 4. Load VGG16 Model:

In [30]:
# IMAGE_SIZE = [224, 224]
vgg = VGG16(weights = 'imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)) 
# include_top = False (where we are defining our own class/label) means we are working at our own dataset where we will have our own label
# here we will have 10 layers (as 10 classes are there)
vgg.output

<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_47>

# 5. Freeze Convolutional Layers (Optional Fine-Tuning):

In [31]:
# we donot need to train our layer, we will work with pre existing weights
for layer in vgg.layers[:10]:
    layer.trainable = False

for layer in vgg.layers[12:]:
    layer.trainable = True
# We have total 19 layers

# 6. Get the Number of Classes from Folder Names:


In [32]:
# folders will contain the all folder under train folder
# folders = glob("/kaggle/input/tomatoleaf/tomato/train/*")
folders = glob(train_dir + '/*')
print(len(folders))

num_classes = len(folders)
print("Number of classes:", num_classes)

10
Number of classes: 10


# 7. Create the Classification Head:

In [33]:
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))                   # Dropout layer
model.add(Dense(256, activation='relu'))  # Additional dense layer
model.add(Dropout(0.3))                   # Dropout layer
model.add(Dense(128, activation='relu'))  # Additional dense layer
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))
print(model)

<Sequential name=sequential_1, built=False>


# 8. Compile the Model (Initial Training):

In [34]:
optimizer = RMSprop(learning_rate=0.0001)  # Consider using a learning rate scheduler
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

## Prepare the data.

# 9. Data Augmentation for Training and Validation:

In [35]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

# 10. Prepare Data Generators:

In [36]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # define directory
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,  # Shuffle the data before splitting
    subset='training',  # Specify that this is for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # Specify that this is for training
)
test_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = 'categorical'
)

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 11. Early Stopping and Learning Rate Reduction:

In [40]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_vgg16_model.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 12. Train the Model (Initial):

In [38]:
epoch = 2
history = model.fit(
    train_generator, epochs=epoch, validation_data=validation_generator, callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/125 ━━━━━━━━━━━━━━━━━━━━ 31:34 15s/step - accuracy: 0.0625 - loss: 2.6538

W0000 00:00:1715681560.511181     104 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.1381 - loss: 2.2855

W0000 00:00:1715681659.083943     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_accuracy improved from -inf to 0.33150, saving model to tomato_vgg16_model.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 138s 990ms/step - accuracy: 0.1385 - loss: 2.2846 - val_accuracy: 0.3315 - val_loss: 1.7548 - learning_rate: 1.0000e-04
Epoch 2/2
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 735ms/step - accuracy: 0.3429 - loss: 1.8552
Epoch 2: val_accuracy improved from 0.33150 to 0.45100, saving model to tomato_vgg16_model.weights.h5
125/125 ━━━━━━━━━━━━━━━━━━━━ 121s 930ms/step - accuracy: 0.3433 - loss: 1.8541 - val_accuracy: 0.4510 - val_loss: 1.5046 - learning_rate: 1.0000e-04


# 13. Evaluate the Model after Initial Training:

In [39]:
score = model.evaluate(validation_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 734ms/step - accuracy: 0.4591 - loss: 1.4912
Test loss (initial): 1.5074516534805298
Test accuracy (initial): 0.45100000500679016


# Predict

# Get Classification Report

In [41]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

 2/16 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step 

W0000 00:00:1715681848.212507     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 223ms/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.92      0.12      0.21       100
                        Tomato___Early_blight       0.31      0.87      0.45       100
                         Tomato___Late_blight       0.91      0.39      0.55       100
                           Tomato___Leaf_Mold       0.56      0.14      0.22       100
                  Tomato___Septoria_leaf_spot       0.46      0.64      0.53       100
Tomato___Spider_mites Two-spotted_spider_mite       0.38      0.45      0.41       100
                         Tomato___Target_Spot       0.49      0.65      0.56       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.99      0.68      0.80       100
                 Tomato___Tomato_mosaic_virus       0.67      0.49      0.57       100
                             Tomato___healthy       0.77      0.77      0.77       100



# 18. Update the weights of the model

In [42]:
model.load_weights('/kaggle/working/tomato_vgg16_model.weights.h5')

## Load Model

In [19]:
# new_vgg

# Test an image for prediction

In [3]:
import numpy as np
import os
from keras.preprocessing import image
from keras.models import load_model

image_path = "/kaggle/input/tomatoleaf/tomato/val/Tomato___Septoria_leaf_spot/0a25f893-1b5f-4845-baa1-f68ac03d96ac___Matt.S_CG 7863.JPG"

# Extract the actual class label from the image path
actual_class = os.path.basename(os.path.dirname(image_path))

img_pred = image.load_img(image_path, target_size=(224,224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0) # adding a dimension

# Define the list of class labels
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']
# passing the array to the model 
# vgg_model = load_model('/kaggle/working/vgg_model_tomato.h5')
# reslt = vgg_model.predict(img_pred) # return a 2d array
reslt = loaded_model.predict(img_pred)
print(reslt)

# Get the predicted class index
predicted_class_index = np.argmax(reslt[0])

# Print the predicted class
predicted_class = class_labels[predicted_class_index]
print("Predicted class:", predicted_class)

# Print the actual class
print("Actual class:", actual_class)

NameError: name 'loaded_model' is not defined